# Day 11

Advent of Code[About][Events][Shop][Settings][Log Out]Andrés Terrer Gómez 15*
   <y>2023</y>[Calendar][AoC++][Sponsors][Leaderboard][Stats]
Our sponsors help make Advent of Code possible:
THE MERGE - The Developer Experience Conference in Berlin, June 2024 (created by the co-founder of GitHub)
--- Day 11: Cosmic Expansion ---
You continue following signs for "Hot Springs" and eventually come across an observatory. The Elf within turns out to be a researcher studying cosmic expansion using the giant telescope here.

He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight there once he's done with today's observation analysis.

Maybe you can help him with the analysis to speed things up?

The researcher has collected a bunch of data and compiled the data into a single giant image (your puzzle input). The image includes empty space (.) and galaxies (#). For example:

...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
The researcher is trying to figure out the sum of the lengths of the shortest path between every pair of galaxies. However, there's a catch: the universe expanded in the time it took the light from those galaxies to reach the observatory.

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.

In the above example, three columns and two rows contain no galaxies:

   v  v  v
 ...#......
 .......#..
 #.........
>..........<
 ......#...
 .#........
 .........#
>..........<
 .......#..
 #...#.....
   ^  ^  ^
These rows and columns need to be twice as big; the result of cosmic expansion therefore looks like this:

....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help to assign every galaxy a unique number:

....1........
.........2...
3............
.............
.............
........4....
.5...........
............6
.............
.............
.........7...
8....9.......
In these 9 galaxies, there are 36 pairs. Only count each pair once; order within the pair doesn't matter. For each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right exactly one . or # at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)

For example, here is one of the shortest paths between galaxies 5 and 9:

....1........
.........2...
3............
.............
.............
........4....
.5...........
.##.........6
..##.........
...##........
....##...7...
8....9.......
This path has length 9 because it takes a minimum of nine steps to get from galaxy 5 to galaxy 9 (the eight locations marked # plus the step onto galaxy 9 itself). Here are some other example shortest path lengths:

Between galaxy 1 and galaxy 7: 15
Between galaxy 3 and galaxy 6: 17
Between galaxy 8 and galaxy 9: 5
In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is 374.

Expand the universe, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

To begin, get your puzzle input.

Answer: 
 

You can also [Share] this puzzle.

In [441]:
# Try everything with the test

puzzle_input = open("./puzzle_inputs/day11test1.txt").read().split("\n")

In [442]:
# Any row or column that contains only "." must be expanded by adding an additional adjacent row/column respectivelly.

# Calculate all pairs of galaxies and get the discrete Manhattan distance between each pair

# First approach:
"""  
    Load the starmap into a np.chararray

    The 2D structure will make it easy to add rows and columns, while also providign with a x-y coordinate system for calculating the manattan distance.
    which whould just be d([x1,y1], [x2,y2]) = abs(x1 - x2) + abs(y1 - y2)
"""

'  \n    Load the starmap into a np.chararray\n\n    The 2D structure will make it easy to add rows and columns, while also providign with a x-y coordinate system for calculating the manattan distance.\n    which whould just be d([x1,y1], [x2,y2]) = abs(x1 - x2) + abs(y1 - y2)\n'

In [443]:
import numpy as np

# Custom functions:


def input2chararray(squaredoc: list[str]) -> np.chararray:
    """
    Given a document with all lines with the same length, create the equivalent np.chararray object
    """

    shape = (len(squaredoc[0]), len(squaredoc))

    arr = np.chararray(shape)

    # Fill the array using the characters from the list
    for i in range(len(squaredoc)):
        for j in range(len(squaredoc[0])):
            arr[j, i] = squaredoc[i][j]

    return arr


def chararray2input(arr: np.chararray) -> list[str]:
    """
    Given a np.chararray, return a list of characters with the proper shape
    """

    # Get the shape of the chararray
    rows, cols = arr.shape

    # Initialize a list to store the characters
    squaredoc = ["" for _ in range(cols)]

    # Fill the list using characters from the chararray
    for i in range(rows):
        for j in range(cols):
            squaredoc[j] += arr[i, j].decode("utf-8")

    return squaredoc


# Re-do the starmap expantion:


def expandStarmap(chararray: np.chararray) -> np.chararray:
    """
    Given a character array:
    duplicate a row if it contains only b"."
    duplicate a column if it contains only b"."
    return the expanded array
    """

    expanded_arr = chararray.copy()

    rows, cols = expanded_arr.shape
    empty_row = ["." for _ in range(cols)]

    # Expand Columns
    added_cols = 0
    for i in range(rows):
        if all([char != b"#" for char in chararray[i, :]]):
            expanded_arr = np.insert(
                arr=expanded_arr, obj=i + added_cols, values=empty_row, axis=0
            )
            added_cols += 1

    # the shape of the arraw has now changed.
    row, cols = expanded_arr.shape
    empty_colum = ["." for _ in range(row)]

    # Expand Rows
    added_rows = 0
    for j in range(cols):
        if all([char != b"#" for char in chararray[:, j]]):
            expanded_arr = np.insert(
                arr=expanded_arr, obj=j + added_rows, values=empty_colum, axis=1
            )
            added_rows += 1

    return expanded_arr


def ManhattanDistance(X: (int, int), Y: (int, int)) -> int:
    """
    Given a pair of points, get their manhattan distance:
    d([x1,y1], [x2,y2]) = abs(x1 - x2) + abs(y1 - y2)
    """

    return abs(X[0] - Y[0]) + abs(X[1] - Y[1])

In [436]:
# Now that we have the expanded starmap, lets add a colletion of all galaxies:
starmap_array = input2chararray(puzzle_input)

expanded_starmap = expandStarmap(starmap_array)

galaxy_dict = {}
rows, cols = expanded_starmap.shape
indx = 1
for i in range(rows):
    for j in range(cols):
        if expanded_starmap[i, j] == b"#":
            galaxy_dict[(i, j)] = indx
            indx += 1

galaxy_dict

{(0, 2): 1,
 (0, 11): 2,
 (1, 6): 3,
 (4, 0): 4,
 (5, 11): 5,
 (8, 5): 6,
 (9, 1): 7,
 (9, 10): 8,
 (12, 7): 9}

In [437]:
# Calculate the distance between pairs of galaxies, avoiding repetition and self-distance (which would be 0 anyways)

galaxy_keys = list(galaxy_dict.keys())

distances = []
for i in range(len(galaxy_keys)):
    for j in range(i + 1, len(galaxy_keys)):
        g1 = galaxy_keys[i]
        g2 = galaxy_keys[j]
        distance = ManhattanDistance(g1, g2)
        distances.append(distance)

print("Galaxies: ", len(distances))
print("Total distance: ", sum(distances))
print("Correct Distance: ", 374)

Galaxies:  36
Total distance:  374
Correct Distance:  374


In [438]:
# Now lets try it with the whole puzzle
puzzle_input = open("./puzzle_inputs/day11.txt").read().split("\n")

starmap_array = input2chararray(puzzle_input)

expanded_starmap = expandStarmap(starmap_array)

galaxy_dict = {}
rows, cols = expanded_starmap.shape
indx = 1
for i in range(rows):
    for j in range(cols):
        if expanded_starmap[i, j] == b"#":
            galaxy_dict[(i, j)] = indx
            indx += 1

galaxy_dict

{(0, 21): 1,
 (0, 34): 2,
 (0, 47): 3,
 (0, 95): 4,
 (0, 116): 5,
 (1, 16): 6,
 (1, 52): 7,
 (1, 105): 8,
 (1, 123): 9,
 (1, 141): 10,
 (2, 30): 11,
 (2, 129): 12,
 (2, 146): 13,
 (3, 12): 14,
 (3, 24): 15,
 (3, 65): 16,
 (4, 6): 17,
 (4, 39): 18,
 (4, 56): 19,
 (4, 119): 20,
 (5, 138): 21,
 (6, 62): 22,
 (6, 95): 23,
 (6, 113): 24,
 (7, 3): 25,
 (7, 16): 26,
 (7, 85): 27,
 (7, 102): 28,
 (8, 50): 29,
 (8, 129): 30,
 (8, 143): 31,
 (9, 30): 32,
 (9, 78): 33,
 (9, 91): 34,
 (10, 22): 35,
 (10, 40): 36,
 (10, 135): 37,
 (11, 14): 38,
 (11, 47): 39,
 (12, 2): 40,
 (12, 66): 41,
 (12, 110): 42,
 (12, 145): 43,
 (13, 59): 44,
 (13, 80): 45,
 (13, 121): 46,
 (13, 132): 47,
 (14, 74): 48,
 (14, 88): 49,
 (14, 115): 50,
 (14, 127): 51,
 (14, 137): 52,
 (15, 24): 53,
 (16, 6): 54,
 (16, 16): 55,
 (17, 45): 56,
 (17, 95): 57,
 (18, 12): 58,
 (18, 20): 59,
 (18, 52): 60,
 (18, 63): 61,
 (18, 125): 62,
 (18, 134): 63,
 (18, 142): 64,
 (19, 1): 65,
 (19, 111): 66,
 (20, 81): 67,
 (21, 28): 68,
 (21

In [440]:
# Calculate the distance between pairs of galaxies, avoiding repetition and self-distance (which would be 0 anyways)

galaxy_keys = list(galaxy_dict.keys())

distances = []
for i in range(len(galaxy_keys)):
    for j in range(i + 1, len(galaxy_keys)):
        g1 = galaxy_keys[i]
        g2 = galaxy_keys[j]
        distance = ManhattanDistance(g1, g2)
        distances.append(distance)

print("Galaxies: ", len(distances))
print("Total distance: ", sum(distances))

Galaxies:  105111
Total distance:  10885634


# Part 2
--- Part Two ---
The galaxies are much older (and thus much farther apart) than the researcher initially estimated.

Now, instead of the expansion you did before, make each empty row or column one million times larger. That is, each empty row should be replaced with 1000000 empty rows, and each empty column should be replaced with 1000000 empty columns.

(In the example above, if each empty row or column were merely 10 times larger, the sum of the shortest paths between every pair of galaxies would be 1030. If each empty row or column were merely 100 times larger, the sum of the shortest paths between every pair of galaxies would be 8410. However, your universe will need to expand far beyond these values.)

Starting with the same initial image, expand the universe according to these new rules, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?



In [ ]:
# First notice:
""" 
Creting an array using the previos method is unfeasable

 - Idea:
    Replace all emplty rows/columns by another character "e"

    Create a function that traverses the manhattan distance between galaxies:
        For each character in its path:
            if its a "." or a "#":
                add +1
            if its a "e":
                add +10**6
            
"""

In [447]:
# Fisrt with test map:

puzzle_input = open("./puzzle_inputs/day11test1.txt").read().split("\n")

In [518]:
# custom function to replace empty rows:


def ReplaceRegions(chararray: np.chararray) -> np.chararray:
    """
    Find all columns/rows that only contain the character b"."
    replace them with "e"
    """

    rows, cols = chararray.shape

    new_arr = chararray.copy()
    empty_col = ["e" for _ in range(rows)]
    empty_row = ["e" for _ in range(cols)]

    for i in range(rows):
        if all([char != b"#" for char in chararray[i, :]]):
            new_arr[i, :] = empty_row

    for j in range(cols):
        if all([char != b"#" for char in chararray[:, j]]):
            new_arr[:, j] = empty_col

    return new_arr


def modifyedMD(starmap: np.chararray, g1: (int, int), g2: (int, int)) -> int:
    """
    given a starmap, travel the manhatan path between galaxy X and Y:
    For each character in its path:
            if its a "." or a "#":
                add +1
            if its a "e":
                add +10**6
    return distance
    """

    x1, y1 = g1[0], g1[1]
    x2, y2 = g2[0], g2[1]

    xstart = min(x1, x2)
    xend = max(x2, x1)

    ystart = min(y1, y2)
    yend = max(y2, y1)

    dist = 0
    for i in range(xstart, xend, 1):
        if starmap[i, ystart] == b"e":
            dist += 10**6
        else:
            dist += 1

    for j in range(ystart, yend, 1):
        if starmap[xend, j] == b"e":
            dist += 10**6
        else:
            dist += 1

    return dist

In [519]:
puzzle_input = open("./puzzle_inputs/day11test1.txt").read().split("\n")
starmap_array = input2chararray(puzzle_input)

expanded_starmap = ReplaceRegions(starmap_array)
chararray2input(expanded_starmap)

['..e#.e..e.',
 '..e..e.#e.',
 '#.e..e..e.',
 'eeeeeeeeee',
 '..e..e#.e.',
 '.#e..e..e.',
 '..e..e..e#',
 'eeeeeeeeee',
 '..e..e.#e.',
 '#.e.#e..e.']

In [520]:
# The same as before, but use the modifyed distance:
galaxy_dict = {}
rows, cols = expanded_starmap.shape
indx = 1
for i in range(rows):
    for j in range(cols):
        if expanded_starmap[i, j] == b"#":
            galaxy_dict[(i, j)] = indx
            indx += 1

galaxy_dict

{(0, 2): 1,
 (0, 9): 2,
 (1, 5): 3,
 (3, 0): 4,
 (4, 9): 5,
 (6, 4): 6,
 (7, 1): 7,
 (7, 8): 8,
 (9, 6): 9}

In [521]:
# Calculate new distance

galaxy_keys = list(galaxy_dict.keys())

distances = []
for i in range(len(galaxy_keys)):
    for j in range(i + 1, len(galaxy_keys)):
        g1 = galaxy_keys[i]
        g2 = galaxy_keys[j]
        distance = modifyedMD(expanded_starmap, g1, g2)
        distances.append(distance)

print("Galaxies: ", len(distances))
print("Total distance: ", sum(distances))

Galaxies:  36
Total distance:  82000210


In [522]:
# Now that it works lets do this with the whole thing:

puzzle_input = open("./puzzle_inputs/day11.txt").read().split("\n")
starmap_array = input2chararray(puzzle_input)

expanded_starmap = ReplaceRegions(starmap_array)

In [523]:
# The same as before, but use the modifyed distance:
galaxy_dict = {}
rows, cols = expanded_starmap.shape
indx = 1
for i in range(rows):
    for j in range(cols):
        if expanded_starmap[i, j] == b"#":
            galaxy_dict[(i, j)] = indx
            indx += 1

# galaxy_dict

In [524]:
galaxy_keys = list(galaxy_dict.keys())

distances = []
for i in range(len(galaxy_keys)):
    for j in range(i + 1, len(galaxy_keys)):
        g1 = galaxy_keys[i]
        g2 = galaxy_keys[j]
        distance = modifyedMD(expanded_starmap, g1, g2)
        distances.append(distance)

print("Galaxies: ", len(distances))
print("Total distance: ", sum(distances))

Galaxies:  105111
Total distance:  707505470642


---
# Day 12

--- Day 12: Hot Springs ---
You finally reach the hot springs! You can see steam rising from secluded areas attached to the primary, ornate building.

As you turn to enter, the researcher stops you. "Wait - I thought you were looking for the hot springs, weren't you?" You indicate that this definitely looks like hot springs to you.

"Oh, sorry, common mistake! This is actually the onsen! The hot springs are next door."

You look in the direction the researcher is pointing and suddenly notice the massive metal helixes towering overhead. "This way!"

It only takes you a few more steps to reach the main gate of the massive fenced-off area containing the springs. You go through the gate and into a small administrative building.

"Hello! What brings you to the hot springs today? Sorry they're not very hot right now; we're having a lava shortage at the moment." You ask about the missing machine parts for Desert Island.

"Oh, all of Gear Island is currently offline! Nothing is being manufactured at the moment, not until we get more lava to heat our forges. And our springs. The springs aren't very springy unless they're hot!"

"Say, could you go up and see why the lava stopped flowing? The springs are too cold for normal operation, but we should be able to find one springy enough to launch you up there!"

There's just one problem - many of the springs have fallen into disrepair, so they're not actually sure which springs would even be safe to use! Worse yet, their condition records of which springs are damaged (your puzzle input) are also damaged! You'll need to help them repair the damaged records.

In the giant field just outside, the springs are arranged into rows. For each row, the condition records show every spring and whether it is operational (.) or damaged (#). This is the part of the condition records that is itself damaged; for some springs, it is simply unknown (?) whether the spring is operational or damaged.

However, the engineer that produced the condition records also duplicated some of this information in a different format! After the list of springs for a given row, the size of each contiguous group of damaged springs is listed in the order those groups appear in the row. This list always accounts for every damaged spring, and each number is the entire size of its contiguous group (that is, groups are always separated by at least one operational spring: #### would always be 4, never 2,2).

So, condition records with no unknown spring conditions might look like this:

#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1
However, the condition records are partially damaged; some of the springs' conditions are actually unknown (?). For example:

???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
Equipped with this information, it is your job to figure out how many different arrangements of operational and broken springs fit the given criteria in each row.

In the first line (???.### 1,1,3), there is exactly one way separate groups of one, one, and three broken springs (in that order) can appear in that row: the first three unknown springs must be broken, then operational, then broken (#.#), making the whole row #.#.###.

The second line is more interesting: .??..??...?##. 1,1,3 could be a total of four different arrangements. The last ? must always be broken (to satisfy the final contiguous group of three broken springs), and each ?? must hide exactly one of the two broken springs. (Neither ?? could be both broken springs or they would form a single contiguous group of two; if that were true, the numbers afterward would have been 2,3 instead.) Since each ?? can either be #. or .#, there are four possible arrangements of springs.

The last line is actually consistent with ten different arrangements! Because the first number is 3, the first and second ? must both be . (if either were #, the first number would have to be 4 or higher). However, the remaining run of unknown spring conditions have many different ways they could hold groups of two and one broken springs:

?###???????? 3,2,1
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
In this example, the number of possible arrangements for each row is:

???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 4 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 1 arrangement
????.######..#####. 1,6,5 - 4 arrangements
?###???????? 3,2,1 - 10 arrangements
Adding all of the possible arrangement counts together produces a total of 21 arrangements.

For each row, count all of the different arrangements of operational and broken springs that meet the given criteria. What is the sum of those counts?



In [23]:
puzzle_input = open("./puzzle_inputs/day12test1.txt").read().split("\n")

In [24]:
class HotspringRow:
    def __init__(self, condition_string: str, condition_list: list[int]):
        self.CondStrig = condition_string
        self.CondList = condition_list


hotspring_list = []
for row in puzzle_input:
    cond_string = row.split(" ")[0]
    cond_list = [int(c) for c in row.split(" ")[1].split(",")]
    item = HotspringRow(condition_list=cond_list, condition_string=cond_string)

    hotspring_list.append(item)


hotspring_list

In [539]:
# Ideas:
""" 
The total number of # is always the sum of the condition list
(from that follows) -> the total number of . is the length of the string - sum(cond_list)
"""

# Approach 1:
""" 
For a given cond_string, construct all possible combinations
Chech for all the combinations that satisfy the cond_list.append:
    Generate the equivalent condition list for all condition subsequences, keep only the ones that are the same as the original sequence.
"""

#

'?'

In [3]:
# Custom class
class HotspringRow:
    def __init__(self, condition_string: str, condition_list: list[int]):
        self.CondStrig = condition_string
        self.CondList = condition_list


# Custom functions:


def ConstructArragements(seq: str) -> list[str]:
    """
    For a given string consisting of "." , "#" and "?" characters
    construct all possible sequences, replacing "?" with "." or "#".
    Lets do this recursivelly:
        Given a string that contains "?":
            return 2 copies of the same string, one swapping it for "#" and another for "."

    """
    subsequences = [seq]

    while any([c == "?" for c in subsequences[0]]):
        new_subsequences = []
        for s in subsequences:
            subseqs = CreateSequences(s)
            new_subsequences.append(subseqs[0])
            new_subsequences.append(subseqs[1])

        subsequences = new_subsequences

    return subsequences


def CreateSequences(seq: str) -> list[str]:
    """
    Given a string that contains "?":
        return 2 copies of the same string, one swapping it for "#" and another for "."
    """

    subsequences = [seq.replace("?", "#", 1), seq.replace("?", ".", 1)]
    return subsequences


def CreateConditionList(seq: str) -> list[int]:
    """
    For a given string of # and ., create a list of integers:
        The list contains the appearences of # in order:

        ".#.##.#...###" -> [1,2,1,3]
        ...
    """
    count = 0
    cd_list = []
    for c in seq:
        if c == "#":
            count += 1
        elif count != 0:
            cd_list.append(count)
            count = 0
    # last number if the sentence ends in "#"
    if seq[-1] == "#":
        cd_list.append(count)

    return cd_list


def CountValidArrangements(hspring: HotspringRow) -> int:
    """
    Given a HotspringRow, return the possible number of arrangemetns that satisfies the list condition
    """
    count = 0
    hspring_string = hspring.CondStrig
    hspring_list = hspring.CondList

    arrangements = ConstructArragements(hspring_string)

    for a in arrangements:
        if CreateConditionList(a) == hspring_list:
            count += 1

    return count

In [4]:
# Put all together:
total_counts = []
for hs in hotspring_list:
    total_counts.append(CountValidArrangements(hs))
print(total_counts)
print(sum(total_counts))

[1, 4, 1, 1, 4, 10]
21


In [620]:
# Now with the whole thing
puzzle_input = open("./puzzle_inputs/day12.txt").read().split("\n")

In [621]:
hotspring_list = []
for row in puzzle_input:
    cond_string = row.split(" ")[0]
    cond_list = [int(c) for c in row.split(" ")[1].split(",")]
    item = HotspringRow(condition_list=cond_list, condition_string=cond_string)

    hotspring_list.append(item)

In [627]:
# Put all together:
total_counts = []
for hs in hotspring_list:
    total_counts.append(CountValidArrangements(hs))
# print(total_counts)
print(sum(total_counts))

7195


# Part 2
--- Part Two ---
As you look out at the field of springs, you feel like there are way more springs than the condition records list. When you examine the records, you discover that they were actually folded up this whole time!

To unfold the records, on each row, replace the list of spring conditions with five copies of itself (separated by ?) and replace the list of contiguous groups of damaged springs with five copies of itself (separated by ,).

So, this row:

.# 1
Would become:

.#?.#?.#?.#?.# 1,1,1,1,1
The first line of the above example would become:

???.###????.###????.###????.###????.### 1,1,3,1,1,3,1,1,3,1,1,3,1,1,3
In the above example, after unfolding, the number of possible arrangements for some rows is now much larger:

???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 16384 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 16 arrangements
????.######..#####. 1,6,5 - 2500 arrangements
?###???????? 3,2,1 - 506250 arrangements
After unfolding, adding all of the possible arrangement counts together produces 525152.

Unfold your condition records; what is the new sum of possible arrangement counts?



In [624]:
# Is it feasable to do this the same way?

""" 
MATH:

The total number of premutations of a repeated sring should be a function of the permutations of each subsring

the thing is that we need to join them using a new "?", so the possible number of strings changes... 
"""

# 1st approach:
""" 
Compute the new possible ways of each string + "?"

The total number of combinations for a 5-fold copy would be (New-combinations) ** 5
"""

In [39]:
# Add the "?" symbol at the end of each string:


def AddMissingSymbol(hotspring: HotspringRow, first=False, last=True):
    """
    Adds a "?" at the end of each condition string
    """
    if last == True:
        hotspring.CondStrig = hotspring.CondStrig + "?"

    if first == True:
        hotspring.CondStrig = "?" + hotspring.CondStrig

    return hotspring

In [50]:
hotspring_list = []
for row in puzzle_input:
    cond_string = row.split(" ")[0]
    cond_list = [int(c) for c in row.split(" ")[1].split(",")]
    item = HotspringRow(condition_list=cond_list, condition_string=cond_string)

    hotspring_list.append(item)


first_hotspring_list = []
for hs in hotspring_list:
    first_hotspring_list.append(AddMissingSymbol(hs, first=False, last=True))

first_counts = []
for hs in first_hotspring_list:
    first_counts.append(CountValidArrangements(hs))
print(first_counts)

hotspring_list = []
for row in puzzle_input:
    cond_string = row.split(" ")[0]
    cond_list = [int(c) for c in row.split(" ")[1].split(",")]
    item = HotspringRow(condition_list=cond_list, condition_string=cond_string)

    hotspring_list.append(item)

middle_hotspring_list = []
for hs in hotspring_list:
    middle_hotspring_list.append(AddMissingSymbol(hs, first=True, last=True))

middle_counts = []
for hs in middle_hotspring_list:
    middle_counts.append(CountValidArrangements(hs))
print(middle_counts)

hotspring_list = []
for row in puzzle_input:
    cond_string = row.split(" ")[0]
    cond_list = [int(c) for c in row.split(" ")[1].split(",")]
    item = HotspringRow(condition_list=cond_list, condition_string=cond_string)

    hotspring_list.append(item)

last_hotspring_list = []
for hs in hotspring_list:
    last_hotspring_list.append(AddMissingSymbol(hs, first=True, last=False))

last_counts = []
for hs in last_hotspring_list:
    last_counts.append(CountValidArrangements(hs))
print(last_counts)


# Compute the total amount of combinations as explained below:

total_counts = [
    first_counts[i] * 3 * middle_counts[i] * last_counts[i]
    for i, __ in enumerate(hotspring_list)
]
print(total_counts)

[1, 4, 1, 1, 4, 15]
[3, 8, 1, 2, 5, 15]
[3, 8, 1, 2, 5, 10]
[27, 768, 3, 12, 300, 6750]


This does not work either ... 

In [47]:
# Re-run the counting of possible arrangements:

total_counts = []
for hs in first_hotspring_list:
    total_counts.append(CountValidArrangements(hs))
print(total_counts)
print(sum(total_counts))

[1, 4, 1, 1, 4, 15]
26


In [29]:
# 5-fold lists counts:

fivefold_counts = [tc**5 for tc in total_counts]
fivefold_counts

[1, 1024, 1, 1, 1024, 759375]

In [ ]:
""" 
This does not produce the correct answer, the total amount os permutations is not calculated this directly, the new ways of creating the arrangements can
include a missing symbol from the last sequence... 

IDEA:

The first sequence must be done using originalString + "?", 
the 3 intermedian string could be done using either previous or following missing symbol,
the last sequence might only be done using the previous "?"

The total number of combinations is the product of the possible combinations:
"Fisrt new string" * 3("Intermedian new strings") * "Last new string"

This does not work either ... 

"""

""" 
New idea:

For every patter find if the new symbol can be one of two options, if it does, replace the next with 2 copies...
"""

# Day 13

--- Day 13: Point of Incidence ---
With your help, the hot springs team locates an appropriate spring which launches you neatly and precisely up to the edge of Lava Island.

There's just one problem: you don't see any lava.

You do see a lot of ash and igneous rock; there are even what look like gray mountains scattered around. After a while, you make your way to a nearby cluster of mountains only to discover that the valley between them is completely full of large mirrors. Most of the mirrors seem to be aligned in a consistent way; perhaps you should head in that direction?

As you move through the valley of mirrors, you find that several of them have fallen from the large metal frames keeping them in place. The mirrors are extremely flat and shiny, and many of the fallen mirrors have lodged into the ash at strange angles. Because the terrain is all one color, it's hard to tell where it's safe to walk or where you're about to run into a mirror.

You note down the patterns of ash (.) and rocks (#) that you see as you walk (your puzzle input); perhaps by carefully analyzing these patterns, you can figure out where the mirrors are!

For example:

#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
To find the reflection in each pattern, you need to find a perfect reflection across either a horizontal line between two rows or across a vertical line between two columns.

In the first pattern, the reflection is across a vertical line between two columns; arrows on each of the two columns point at the line between the columns:

123456789
    ><   
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
    ><   
123456789
In this pattern, the line of reflection is the vertical line between columns 5 and 6. Because the vertical line is not perfectly in the middle of the pattern, part of the pattern (column 1) has nowhere to reflect onto and can be ignored; every other column has a reflected column within the pattern and must match exactly: column 2 matches column 9, column 3 matches 8, 4 matches 7, and 5 matches 6.

The second pattern reflects across a horizontal line instead:

1 #...##..# 1
2 #....#..# 2
3 ..##..### 3
4v#####.##.v4
5^#####.##.^5
6 ..##..### 6
7 #....#..# 7
This pattern reflects across the horizontal line between rows 4 and 5. Row 1 would reflect with a hypothetical row 8, but since that's not in the pattern, row 1 doesn't need to match anything. The remaining rows match: row 2 matches row 7, row 3 matches row 6, and row 4 matches row 5.

To summarize your pattern notes, add up the number of columns to the left of each vertical line of reflection; to that, also add 100 multiplied by the number of rows above each horizontal line of reflection. In the above example, the first pattern's vertical line has 5 columns to its left and the second pattern's horizontal line has 4 rows above it, a total of 405.

Find the line of reflection in each of the patterns in your notes. What number do you get after summarizing all of your notes?

In [88]:
""" 
IDEA for finding reflections:

for every array find:

row compasiron array, rowarray = [row_i == row_j]

find all possible minors for every diagonal element
if any of those have an all True anti-diagonal, then the reflection index is i + shape of the minor /2, this will allways happend when the shape is even so we
can kip odd shapes.

We only take the minors that contain the element (0,0) or (s,s) !! otherwise we are taking sub-patterns that do not end at the boundary
"""

In [333]:
# Custom functions:


def getPatterns(puzzle_input: list[str]) -> list[list[str]]:
    """
    Splits the puzzle input into chunks of retangular maps with . and # symbols.
    """

    pattern_list = []
    pattern = []

    for line in puzzle_input:
        if line != "":
            pattern.append(line)
        else:
            pattern_list.append(pattern)
            pattern = []

    # append the last pattern
    pattern_list.append(pattern)
    return pattern_list


# Re-use functions to get the chararray from the string list:
import numpy as np


def input2chararray(squaredoc: list[str]) -> np.chararray:
    """
    Given a document with all lines with the same length, create the equivalent np.chararray object
    """

    shape = (len(squaredoc[0]), len(squaredoc))

    arr = np.chararray(shape)

    # Fill the array using the characters from the list
    for i in range(len(squaredoc)):
        for j in range(len(squaredoc[0])):
            arr[j, i] = squaredoc[i][j]

    return arr


def chararray2input(arr: np.chararray) -> list[str]:
    """
    Given a np.chararray, return a list of characters with the proper shape
    """

    # Get the shape of the chararray
    rows, cols = arr.shape

    # Initialize a list to store the characters
    squaredoc = ["" for _ in range(cols)]

    # Fill the list using characters from the chararray
    for i in range(rows):
        for j in range(cols):
            squaredoc[j] += arr[i, j].decode("utf-8")

    return squaredoc


# Specific for this puzzle:


def anti_diagonal(arr: np.array) -> np.array:
    """
    Returns the anti-diagonal of an array.
    """
    return np.fliplr(arr).diagonal()


def FindTrueMinors(boolarr: np.array) -> int:
    """
    Look at the relevant minors in the boolean arrays and
    return the index where a simmetry occurs
    """

    # look for minors that contains the (0,0) element
    s, __ = boolarr.shape

    for i in range(0, s + 1, 2):
        minor = boolarr[0 : i + 2, 0 : i + 2]

        if np.all(anti_diagonal(minor)):
            return i + 2 - minor.shape[0] // 2

    # Look for minors that contain the (s,s) element
    for i in range(s - 2, 0, -2):
        minor = boolarr[i : s + 1, i : s + 1]
        if np.all(anti_diagonal(minor)):
            return i + minor.shape[0] // 2

    return -1


def BooleanArray(arr: np.array, how) -> np.array:
    """
    Creates the boolean array
    """

    if how == "cols":
        num = arr.shape[0]
        b_array = np.full((num, num), False, dtype=bool)

        for i in range(num):
            for j in range(num):
                b_array[i, j] = np.all(arr[i] == arr[j])

        return b_array

    if how == "rows":
        num = arr.shape[1]
        b_array = np.full((num, num), False, dtype=bool)

        for i in range(num):
            for j in range(num):
                b_array[i, j] = np.all(arr[:, i] == arr[:, j])

        return b_array


def FindReflexion(arr: np.chararray) -> tuple[int, int]:
    """
    Calculate the columns and rows boolean values

    check for minors with all True antidiagonals

    return the index and save it into the reflexion plane (row index, col index)

    """
    # Define Boolean Columns array
    b_cols = BooleanArray(arr, how="cols")

    # Define Boolean Rows array
    b_rows = BooleanArray(arr, how="rows")

    # Find the index (if any) of the reflexion using minors:

    reflexion_plane = (FindTrueMinors(b_rows), FindTrueMinors(b_cols))

    return reflexion_plane

In [334]:
# TEST
puzzle_input = open("./puzzle_inputs/day13test.txt").read().split("\n")

pattern_list = getPatterns(puzzle_input)
chararray_list = [input2chararray(p) for p in pattern_list]

# Create a list with the reflexions
reflexion_list = [FindReflexion(charr) for charr in chararray_list]

# Compute the summary
summary = 0

for r in reflexion_list:
    row, col = r
    if row > 0:
        summary += 100 * row
    if col > 0:
        summary += col
print(summary)

405


In [335]:
# Now with the whole thing:
puzzle_input = open("./puzzle_inputs/day13.txt").read().split("\n")

pattern_list = getPatterns(puzzle_input)
chararray_list = [input2chararray(p) for p in pattern_list]

reflexion_list = [FindReflexion(charr) for charr in chararray_list]

summary = 0
for r in reflexion_list:
    row, col = r
    if row > 0:
        summary += 100 * row
    if col > 0:
        summary += col
print(summary)

33356


# PART 2

--- Part Two ---
You resume walking through the valley of mirrors and - SMACK! - run directly into one. Hopefully nobody was watching, because that must have been pretty embarrassing.

Upon closer inspection, you discover that every mirror has exactly one smudge: exactly one . or # should be the opposite type.

In each pattern, you'll need to locate and fix the smudge that causes a different reflection line to be valid. (The old reflection line won't necessarily continue being valid after the smudge is fixed.)

Here's the above example again:

#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
The first pattern's smudge is in the top-left corner. If the top-left # were instead ., it would have a different, horizontal line of reflection:

1 ..##..##. 1
2 ..#.##.#. 2
3v##......#v3
4^##......#^4
5 ..#.##.#. 5
6 ..##..##. 6
7 #.#.##.#. 7
With the smudge in the top-left corner repaired, a new horizontal line of reflection between rows 3 and 4 now exists. Row 7 has no corresponding reflected row and can be ignored, but every other row matches exactly: row 1 matches row 6, row 2 matches row 5, and row 3 matches row 4.

In the second pattern, the smudge can be fixed by changing the fifth symbol on row 2 from . to #:

1v#...##..#v1
2^#...##..#^2
3 ..##..### 3
4 #####.##. 4
5 #####.##. 5
6 ..##..### 6
7 #....#..# 7
Now, the pattern has a different horizontal line of reflection between rows 1 and 2.

Summarize your notes as before, but instead use the new different reflection lines. In this example, the first pattern's new horizontal line has 3 rows above it and the second pattern's new horizontal line has 1 row above it, summarizing to the value 400.

In each pattern, fix the smudge and find the different line of reflection. What number do you get after summarizing the new reflection line in each pattern in your notes?




In [ ]:
""" 
First idea: Brute force it

compute all possible permutations and get their reflexion, if the reflexion line is different, then shwap it.
"""